In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic2/train.csv
/kaggle/input/titanic2/test.csv
/kaggle/input/titanic2/gender_submission.csv


In [2]:
# 1. Cargar los datos
train_data = pd.read_csv('/kaggle/input/titanic2/train.csv')  # Asegúrate de poner la ruta correcta al archivo original
test_data = pd.read_csv('/kaggle/input/titanic2/test.csv')

# 2. Revisar la información de los datos
print(train_data.info())
print(test_data.info())

# 3. Imputar los valores faltantes de 'Age'
train_data['Age'] = pd.to_numeric(train_data['Age'], errors='coerce')  # Convertir 'Age' a numérico y manejar errores
test_data['Age'] = pd.to_numeric(test_data['Age'], errors='coerce')

# Imputar los valores faltantes en 'Age' con la mediana
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)

# 4. Imputar valores faltantes de 'Embarked'
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
test_data['Embarked'].fillna(test_data['Embarked'].mode()[0], inplace=True)

# 5. Manejo de la columna 'Fare'
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

# 6. Crear la columna 'Cabin_known'
train_data['Cabin_known'] = train_data['Cabin'].notnull().astype(int)
test_data['Cabin_known'] = test_data['Cabin'].notnull().astype(int)

# Eliminar la columna 'Cabin'
train_data.drop(columns=['Cabin'], inplace=True)
test_data.drop(columns=['Cabin'], inplace=True)

# 7. Convertir la columna 'Sex' en numérica (One-Hot Encoding)
train_data = pd.get_dummies(train_data, columns=['Sex'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Sex'], drop_first=True)

# 8. Convertir la columna 'Embarked' en variables numéricas (One-Hot Encoding)
train_data = pd.get_dummies(train_data, columns=['Embarked'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Embarked'], drop_first=True)

# 9. Eliminar columnas irrelevantes ('Name', 'Ticket', 'PassengerId')
train_data.drop(columns=['Name', 'Ticket', 'PassengerId'], inplace=True)
test_data.drop(columns=['Name', 'Ticket', 'PassengerId'], inplace=True)

# 10. Convertir valores True/False en 0/1 (para las columnas booleanas)
train_data = train_data.astype({col: 'int' for col in train_data.select_dtypes(include='bool').columns})
test_data = test_data.astype({col: 'int' for col in test_data.select_dtypes(include='bool').columns})

# Verificar las primeras filas del dataset después de las transformaciones
print(train_data.head())
print(test_data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcl

/tmp/ipykernel_31/687043411.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
/tmp/ipykernel_31/687043411.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [19]:
# Separar las características (X) y el target (y) en el conjunto de entrenamiento
X_train=train_data.drop(columns=['Survived'])
y_train=train_data['Survived']
X_test=test_data

In [29]:
# Verificar las dimensiones de los datos
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (891, 9)
y_train shape: (891,)
X_test shape: (418, 9)


In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Crear el modelo Random Forest con 65 árboles y una profundidad máxima de 15
model = RandomForestClassifier(n_estimators=65,max_depth=15,random_state=42)

# Aplicar validación cruzada con 5 folds para evaluar el rendimiento del modelo
scores=cross_val_score(model,X_train,y_train,cv=5,scoring='accuracy')
# Mostrar la precisión en cada fold y la precisión promedio
print(f"Precisión en cada fold: {scores}")
print(f"Precisión promedio en validación cruzada: {scores.mean():.4f}")

Precisión en cada fold: [0.79888268 0.78651685 0.85955056 0.78651685 0.83146067]
Precisión promedio en validación cruzada: 0.8126


In [68]:
# Dividir los datos de entrenamiento en un conjunto de entrenamiento (80%) y validación (20%)
X_train_2,X_val,y_train_2,y_val=train_test_split(X_train, y_train, test_size=0.2,random_state=42)

# Entrenar el modelo en los datos de entrenamiento
model.fit(X_train_2,y_train_2)

# Hacer predicciones en el conjunto de validación
y_predict=model.predict(X_val)

# Evaluar el modelo usando el conjunto de validación 
scores=accuracy_score(y_val,y_predict)

# Mostrar la precisión del modelo en el conjunto de validación
print(f"Precision: {scores:.4f}")

Precision: 0.8212


In [76]:
# Hacer predicciones en el conjunto de prueba
y_pred_test=model.predict(X_test)

# Crear un DataFrame con 'PassengerId' y las predicciones 'Survived'
output = pd.DataFrame({'PassengerId': test_data.index+892 , 'Survived': y_pred_test})

# Guardar las predicciones en un archivo CSV para subir a Kaggle
output.to_csv('titanic_predictions.csv', index=False)
output.tail()


,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,0
